Read monthly precipitation from the CMIP5 models Historical simulations and puts them all in the same grid and time period in a python-friendly structure (dictionary or xarray). 

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (8,5)

import cartopy.crs as ccrs

from glob import glob

import os, sys
sys.path.append("/home2/biasutti/py_packages")  # substitute your username for nhn2
from ingrid import times
from ingrid.times import to_pandas
from ingrid.times import enso2date

In [2]:
models = !ls /net/strega.ldeo.columbia.edu/home/CMIP5/byScenario/historical/atmos/mon/pr


In [9]:
path = '/net/strega.ldeo.columbia.edu/home/CMIP5/byScenario/historical/atmos/mon/pr'
dirs = glob(path+'/*')
files = glob(path+'/*/r1i1p1/pr.nc')
ds =  list(map(lambda x: xr.open_dataset(x,decode_times=False), files))

In [4]:
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-89,89, 2)),
                     'lon': (['lon'], np.arange(-179,179,2)),
                    }
                   )
regridder = list(map(lambda x: 
                xe.Regridder(x, ds_out, 'bilinear',
                periodic=True, reuse_weights=True ), ds))

Reuse existing file: bilinear_48x96_89x179_peri.nc
Reuse existing file: bilinear_145x192_89x179_peri.nc
Reuse existing file: bilinear_90x144_89x179_peri.nc
Reuse existing file: bilinear_64x128_89x179_peri.nc
Reuse existing file: bilinear_90x144_89x179_peri.nc
Reuse existing file: bilinear_128x256_89x179_peri.nc
Reuse existing file: bilinear_96x192_89x179_peri.nc
Reuse existing file: bilinear_160x320_89x179_peri.nc
Reuse existing file: bilinear_320x640_89x179_peri.nc
Reuse existing file: bilinear_96x144_89x179_peri.nc
Reuse existing file: bilinear_96x144_89x179_peri.nc
Reuse existing file: bilinear_160x320_89x179_peri.nc
Reuse existing file: bilinear_90x144_89x179_peri.nc
Reuse existing file: bilinear_145x192_89x179_peri.nc
Reuse existing file: bilinear_64x128_89x179_peri.nc
Reuse existing file: bilinear_64x128_89x179_peri.nc
Reuse existing file: bilinear_64x128_89x179_peri.nc
Reuse existing file: bilinear_96x96_89x179_peri.nc
Reuse existing file: bilinear_90x144_89x179_peri.nc
Reuse ex

In [12]:
# rewrite time in pandas format

i = 0
for k in ds:
    #print(k.time.units)
    #print(k.model_id)
    k['time'] = times.to_pandas(k['time'])
    i += 1




In [10]:
# this does not work... is there a way to do the loop above this way?
#ds_out = list(map(lambda x: to_pandas(x['time']), ds))
#ds_out

In [18]:
#pr = list(map(lambda x: x.pr.sel(time=slice('1901-01-01', '2001-01-01')),ds))
#pr_2x2 = list(map(lambda x,y: x(y),regridder,pr))
pr_2x2 = list(map(lambda x,y: 
                  x(y.pr.sel(time=slice('1901-01-01', '2001-01-01'))),
                  regridder,ds))


In [14]:
# make a dictionary
ensemble = {'model':models,'data':pr_2x2}
# or make a xarray with an extra dimension
pr_ens = xr.concat(pr_2x2[:],dim='model')